In [262]:
import sys
import os
import pandas as pd
import random

sys.path.append(os.path.abspath('../src'))

from models import Player, Team, Individual
from utils import create_valid_team, create_valid_individual, generate_initial_population
from crossover import (
    team_swap_crossover,
    player_swap_crossover,
    uniform_team_crossover,
    position_based_crossover,
    gene_level_crossover
)
from mutation import (
    random_player_mutation,
    swap_players_between_teams,
    replace_team_mutation,
    expensive_player_mutation,
    cross_position_mutation,
    forward_reset_mutation,
    rotate_teams_mutation
)
from selection import (
    tournament_selection,
    roulette_wheel_selection,
    truncation_selection,
    rank_selection,
    stochastic_universal_sampling,
    elitism,
    select_parents
)

from main import run_ga_from_notebook


In [263]:
players_csv = pd.read_csv('../data/players.csv')

In [264]:
players = []
for idx, row in players_csv.iterrows():
    p = Player(
        player_id=idx + 1,
        name=row['Name'],
        position=row['Position'],
        skill=row['Skill'],
        cost=row['Salary (€M)']
    )

    players.append(p)

print(f"Número de jogadores carregados: {len(players)}")

Número de jogadores carregados: 35


In [265]:
used_ids = set()
team = create_valid_team(players, used_ids)

if team:
    print([str(p) for p in team.players])
    print("Válida:", team.is_valid())
    print("Custo total:", team.total_cost())
    print("Skill média:", team.average_skill())
else:
    print("⚠️ Não foi possível gerar uma equipa válida.")

['Blake Henderson (GK) - Skill: 87 | €M95', 'Lucas Bennett (DEF) - Skill: 85 | €M90', 'Daniel Foster (DEF) - Skill: 90 | €M110', 'Gavin Richardson (MID) - Skill: 87 | €M95', 'Ashton Phillips (MID) - Skill: 90 | €M110', 'Elijah Sanders (FWD) - Skill: 93 | €M140', 'Adrian Collins (FWD) - Skill: 85 | €M90']
Válida: True
Custo total: 730
Skill média: 88.14285714285714


In [266]:
ind = create_valid_individual(players)
if ind is not None:
    print("Indivíduo válido: ", ind.is_valid())
    print("Fitness (desvio padrão negativo):", ind.fitness())
else:
    print("⚠️ Não foi possível criar um indivíduo válido.")

Indivíduo válido:  True
Fitness (desvio padrão negativo): -1.0781692722350371


In [267]:
parent1 = create_valid_individual(players)
parent2 = create_valid_individual(players)

if not parent1 or not parent2:
    print("Erro: não foi possível gerar pais válidos.")
else:
    print("Pais criados com sucesso.")


Pais criados com sucesso.


In [268]:
child1, child2 = team_swap_crossover(parent1, parent2)
print("Team Swap -> Fitness Child1:", child1.fitness(), "| Child2:", child2.fitness())
print("Child1 válido?", child1.is_valid())
print("Child2 válido?", child2.is_valid())

Team Swap -> Fitness Child1: -1.240144823669182 | Child2: -0.6663945022680319
Child1 válido? True
Child2 válido? True


In [269]:
child1, child2 = player_swap_crossover(parent1, parent2)
print("Player Swap -> Fitness Child1:", child1.fitness(), "| Child2:", child2.fitness())
print("Child1 válido?", child1.is_valid())
print("Child2 válido?", child2.is_valid())

Player Swap -> Fitness Child1: -1.332789004536067 | Child2: -0.5952190473142756
Child1 válido? True
Child2 válido? True


In [270]:
child = uniform_team_crossover(parent1, parent2)
print("Uniform Crossover -> Fitness:", child.fitness())
print("Válido:", child.is_valid())

Uniform Crossover -> Fitness: -1.2506324930422437
Válido: True


In [271]:
child = position_based_crossover(parent1, parent2)
print("Position-Based Crossover -> Fitness:", child.fitness())
print("Válido:", child.is_valid())

Position-Based Crossover -> Fitness: -1.3018040857022322
Válido: False


In [272]:
child = gene_level_crossover(parent1, parent2)
print("Gene-Level Crossover -> Fitness:", child.fitness())
print("Válido:", child.is_valid())

Gene-Level Crossover -> Fitness: -2.074923160613178
Válido: False


In [273]:
individual_mut = create_valid_individual(players)
if individual_mut is None:
    print("⚠️ Não foi possível gerar um indivíduo válido.")
else:
    print("Indivíduo base gerado com sucesso.")

Indivíduo base gerado com sucesso.


In [274]:
mutated = random_player_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1117186547624969
Válido: True


In [275]:
mutated = swap_players_between_teams(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1299268533305802
Válido: True


In [276]:
mutated = replace_team_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1117186547624969
Válido: True


In [277]:
mutated = expensive_player_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1117186547624969
Válido: True


In [278]:
mutated = cross_position_mutation(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1263087625386854
Válido: True


In [279]:
mutated = forward_reset_mutation(individual_mut, players)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1117186547624969
Válido: True


In [280]:
mutated = rotate_teams_mutation(individual_mut)
print("Antes:", individual_mut.fitness())
print("Depois:", mutated.fitness())
print("Válido:", mutated.is_valid())

Antes: -1.1117186547624969
Depois: -1.1117186547624969
Válido: True


In [281]:
population = generate_initial_population(10, players)
fitnesses = [ind.fitness() for ind in population]

In [282]:
parent = tournament_selection(population, fitnesses)
print("Torneio -> Fitness:", parent.fitness())

Torneio -> Fitness: -1.0435418495856956


In [283]:
parent = roulette_wheel_selection(population, fitnesses)
print("Roleta -> Fitness:", parent.fitness())

Roleta -> Fitness: -1.4439175735420902


In [284]:
parent = truncation_selection(population, fitnesses, top_percent=0.3)
print("Truncamento -> Fitness:", parent.fitness())

Truncamento -> Fitness: -0.6845227743263387


In [285]:
parent = stochastic_universal_sampling(population, fitnesses)
print("SUS -> Fitness:", parent.fitness())

SUS -> Fitness: -0.6845227743263387


In [286]:
elites = elitism(population, fitnesses, num_elites=2)
print("Elites -> Fitnesses:", [e.fitness() for e in elites])

Elites -> Fitnesses: [-0.6845227743263387, -0.7740695533796581]


In [287]:
p1, p2 = select_parents(population, fitnesses, method="tournament")
print("select_parents (Torneio) ->", p1.fitness(), p2.fitness())

select_parents (Torneio) -> -1.0513353995985193 -1.1654901745354942
